# Distributing Jupyter Extensions as Python Packages

## Overview
### How can the notebook be extended?
The Jupyter Notebook client and server application are both deeply customizable. Their behavior can be extended by creating, respectively:

- nbextension: a notebook extension
    - a single JS file, or directory of JavaScript, Cascading StyleSheets, etc. that contain at
      minimum a JavaScript module packaged as an
      [AMD modules](https://en.wikipedia.org/wiki/Asynchronous_module_definition)
      that exports a function `load_ipython_extension`
- server extension: an importable Python module
    - that implements `load_jupyter_server_extension`
- bundler extension: an importable Python module with generated File -> Download as / Deploy as menu item trigger
    - that implements `bundle`

### Why create a Python package for Jupyter extensions?
Since it is rare to have a server extension that does not have any frontend components (an nbextension), for convenience and consistency, all these client and server extensions with their assets can be packaged and versioned together as a Python package with a few simple commands. This makes installing the package of extensions easier and less error-prone for the user. 

## Installation of Jupyter Extensions
### Install a Python package containing Jupyter Extensions
There are several ways that you may get a Python package containing Jupyter Extensions. Commonly, you will use a package manager for your system:
```shell
pip install helpful_package
# or
conda install helpful_package
# or
apt-get install helpful_package

# where 'helpful_package' is a Python package containing one or more Jupyter Extensions
```

### Enable a Server Extension

The simplest case would be to enable a server extension which has no frontend components. 

A `pip` user that wants their configuration stored in their home directory would type the following command:
```shell
jupyter serverextension enable --py helpful_package
```

Alternatively, a `virtualenv` or `conda` user can pass `--sys-prefix` which keeps their environment isolated and reproducible. For example:
```shell
# Make sure that your virtualenv or conda environment is activated
[source] activate my-environment

jupyter serverextension enable --py helpful_package --sys-prefix
```

### Install the nbextension assets

If a package also has an nbextension with frontend assets that must be available (but not neccessarily enabled by default), install these assets with the following command:
```shell
jupyter nbextension install --py helpful_package # or --sys-prefix if using virtualenv or conda
```

### Enable nbextension assets
If a package has assets that should be loaded every time a Jupyter app (e.g. lab, notebook, dashboard, terminal) is loaded in the browser, the following command can be used to enable the nbextension:
```shell
jupyter nbextension enable --py helpful_package # or --sys-prefix if using virtualenv or conda
```

## Did it work? Check by listing Jupyter Extensions.
After running one or more extension installation steps, you can list what is presently known about nbextensions, server extensions, or bundler extensions. The following commands will list which extensions are available, whether they are enabled, and other extension details:

```shell
jupyter nbextension list
jupyter serverextension list
jupyter bundlerextension list
```

## Additional resources on creating and distributing packages        

> Of course, in addition to the files listed, there are number of other files one needs to build a proper package. Here are some good resources:
- [The Hitchhiker's Guide to Packaging](https://the-hitchhikers-guide-to-packaging.readthedocs.io/en/latest/quickstart.html)
- [Repository Structure and Python](https://www.kennethreitz.org/essays/repository-structure-and-python) by Kenneth Reitz

> How you distribute them, too, is important:
- [Packaging and Distributing Projects](https://python-packaging-user-guide.readthedocs.io/tutorials/distributing-packages/)
- [conda: Building packages](https://conda.io/docs/building/build.html)

> Here are some tools to get you started:
- [generator-nbextension](https://github.com/Anaconda-Platform/generator-nbextension)

## Example - Server extension

### Creating a Python package with a server extension

Here is an example of a python module which contains a server extension directly on itself. It has this directory structure:
```
- setup.py
- MANIFEST.in
- my_module/
  - __init__.py
```

### Defining the server extension
This example shows that the server extension and its `load_jupyter_server_extension` function are defined in the `__init__.py` file.

#### `my_module/__init__.py`

```python
def _jupyter_server_extension_paths():
    return [{
        "module": "my_module"
    }]


def load_jupyter_server_extension(nbapp):
    nbapp.log.info("my module enabled!")
```

### Install and enable the server extension
Which a user can install with:
```
jupyter serverextension enable --py my_module [--sys-prefix]
```

## Example - Server extension and nbextension

### Creating a Python package with a server extension and nbextension
Here is another server extension, with a front-end module.  It assumes this directory structure:

```
- setup.py
- MANIFEST.in
- my_fancy_module/
  - __init__.py
  - static/
    index.js
```

### Defining the server extension and nbextension
This example again shows that the server extension and its `load_jupyter_server_extension` function are defined in the `__init__.py` file. This time, there is also a function `_jupyter_nbextension_path` for the nbextension.

#### `my_fancy_module/__init__.py`

```python
def _jupyter_server_extension_paths():
    return [{
        "module": "my_fancy_module"
    }]

# Jupyter Extension points
def _jupyter_nbextension_paths():
    return [dict(
        section="notebook",
        # the path is relative to the `my_fancy_module` directory
        src="static",
        # directory in the `nbextension/` namespace
        dest="my_fancy_module",
        # _also_ in the `nbextension/` namespace
        require="my_fancy_module/index")]

def load_jupyter_server_extension(nbapp):
    nbapp.log.info("my module enabled!")
```

### Install and enable the server extension and nbextension

The user can install and enable the extensions with the following set of commands:
```
jupyter nbextension install --py my_fancy_module [--sys-prefix|--user]
jupyter nbextension enable --py my_fancy_module [--sys-prefix|--system]
jupyter serverextension enable --py my_fancy_module [--sys-prefix|--system]
```

## Example - Bundler extension

### Creating a Python package with a bundlerextension

Here is a bundler extension that adds a *Download as -> Notebook Tarball (tar.gz)* option to the notebook *File* menu. It assumes this directory structure:

```
- setup.py
- MANIFEST.in
- my_tarball_bundler/
  - __init__.py
```

### Defining the bundler extension

This example shows that the bundler extension and its `bundle` function are defined in the `__init__.py` file.

#### `my_tarball_bundler/__init__.py`

```python
import tarfile
import io
import os
import nbformat

def _jupyter_bundlerextension_paths():
    """Declare bundler extensions provided by this package."""
    return [{
        # unique bundler name
        "name": "tarball_bundler",
        # module containing bundle function
        "module_name": "my_tarball_bundler",
        # human-redable menu item label
        "label" : "Notebook Tarball (tar.gz)",
        # group under 'deploy' or 'download' menu
        "group" : "download",
    }]


def bundle(handler, model):
    """Create a compressed tarball containing the notebook document.
    
    Parameters
    ----------
    handler : tornado.web.RequestHandler
        Handler that serviced the bundle request
    model : dict
        Notebook model from the configured ContentManager
    """
    notebook_filename = model['name']
    notebook_content = nbformat.writes(model['content']).encode('utf-8')
    notebook_name = os.path.splitext(notebook_filename)[0]
    tar_filename = '{}.tar.gz'.format(notebook_name)
    
    info = tarfile.TarInfo(notebook_filename)
    info.size = len(notebook_content)

    with io.BytesIO() as tar_buffer:
        with tarfile.open(tar_filename, "w:gz", fileobj=tar_buffer) as tar:
            tar.addfile(info, io.BytesIO(notebook_content))
        
        # Set headers to trigger browser download
        handler.set_header('Content-Disposition',
                           'attachment; filename="{}"'.format(tar_filename))
        handler.set_header('Content-Type', 'application/gzip')
                
        # Return the buffer value as the response
        handler.finish(tar_buffer.getvalue())
```

See [Extending the Notebook](../../extending/index.rst) for more documentation about writing nbextensions, server extensions, and bundler extensions.